In [15]:
#CODE WOKWI NẰM MÚT DƯỚI CÙNG!!!!!!!!!!!!!!!!!!!!!!!

Note: you may need to restart the kernel to use updated packages.


'"c:\Users\THANH TAI\AppData\Local\Programs\Python\Python312\python.exe"' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes
import argparse
import imutils
import pickle
import cv2
import os
from PIL import Image 


In [48]:
model = load_model(r'E:\Nghe\20004175_VuThanhTai\output\20004175_model.h5')
from keras.preprocessing.image import img_to_array

image_name = r'beetle (419).jpg'
folder_path = r'Test images'

# Kết hợp đường dẫn thư mục và tên hình ảnh
image_path = os.path.join(folder_path, image_name)

# Mở hình ảnh
image = Image.open(image_path)
image = image.resize((112, 112))
image = img_to_array(image) / 255.0
image = np.expand_dims(image, axis=0)
(boxPreds, labelPreds) = model.predict(image)
lb = ['beetle', 'grasshopper', 'mango_hopper', 'mango_mealybug', 'moth']
i = np.argmax(labelPreds, axis=1)
result = lb[np.argmax(labelPreds)]
accuracy = float(labelPreds[0][i]) * 100
print(accuracy)
print("------")
print(result)
if (result=="beetle"):
    result = 1
if (result=="grasshopper"):
    result = 2
if (result=="mango_hopper"):
    result = 3
if (result=="mango_mealybug"):
    result = 4
if (result=="moth"):
    result = 5
print("------")
print(result)

1/1 [==============================] - 3s 3s/step
100.0
------
beetle
------
1


C:\Users\THANH TAI\AppData\Local\Temp\ipykernel_25468\232350072.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  accuracy = float(labelPreds[0][i]) * 100


In [42]:
import mysql.connector
db = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = "",
  database = "plant_disease"
)
if db.is_connected():
    print("Ket noi thanh cong")
cursor = db.cursor()


Ket noi thanh cong


In [59]:
import paho.mqtt.client as mqtt
sql = f"SELECT d.name, d.is_danger FROM disease d JOIN images i ON d.id_disease = i.id_disease WHERE i.file_name = '{image_name}'"
cursor.execute(sql)
count =  cursor.fetchall()
for row in count:
    name = row[0]
    status = row[1]
    print("name :", name)
    print("status :", status)


name : 1
status : 1


In [68]:
import paho.mqtt.client as mqtt
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Kết nối thành công")
    else:
        print("Kết nối thất bại")
def on_publish(client, userdata, mid):
    print("Đã gửi dữ liệu ")

client = mqtt.Client()
client.on_connect = on_connect
client.on_publish = on_publish
client.connect("broker.emqx.io", 1883)
if status == 0:
    client.publish("tai", "khong nguy hiem")
else:
     client.publish("tai", "nguy hiem")
     client.publish("tai", "TT" + name + status)

Đã gửi dữ liệu 


TypeError: can only concatenate str (not "int") to str

In [ ]:
#include <WiFi.h>
#include <Wire.h>
#include <LiquidCrystal_I2C.h>
#include <Adafruit_GFX.h>
#include <Adafruit_SSD1306.h>
#include <ArduinoJson.h>
#include <PubSubClient.h>
#include "DHTesp.h"
#define SCREEN_WIDTH 128
#define SCREEN_HEIGHT 64

Adafruit_SSD1306 oled(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, -1); //OLED
LiquidCrystal_I2C lcd (0x27, 20, 4); //LCD
float temp, hum; //2 bien nhiet do, do am
const int ledPin = 13; //khai bao chan led
const int buzz = 12; //khai bao chan buzzer
const int relay = 14; //khai bao chan relay
const int ldr = 35; //khai bao chan ldr
const int DHT_PIN = 15; //Khai bao chan dht22
DHTesp dht;

const char * MQTTServer = "broker.emqx.io";
const char * MQTT_Topic = "tai";

// Tạo ID ngẫu nhiên tại: https://www.guidgen.com/
const char * MQTT_ID = "";
int Port = 1883;

WiFiClient espClient; //Khai bao esp wifi
PubSubClient client(espClient);
// *Ket noi MQTT
void WIFIConnect() {
  Serial.println("Connecting to SSID: Wokwi-GUEST");
  WiFi.begin("Wokwi-GUEST", "");
  while (WiFi.status() != WL_CONNECTED) {
    delay(500);
    Serial.print(".");
  }
  Serial.println("");
  Serial.print("WiFi connected");
  Serial.print(", IP address: ");
  Serial.println(WiFi.localIP());
}

void MQTT_Reconnect() {
  while (!client.connected()) {
    if (client.connect(MQTT_ID)) {
      Serial.print("MQTT Topic: ");
      Serial.print(MQTT_Topic);
      Serial.print("connected");
      client.subscribe(MQTT_Topic);
      Serial.println("");
    } else {
      Serial.print("failed, rc=");
      Serial.print(client.state());
      Serial.println(" try again in 5 seconds");
      delay(5000);
    }
  }
}
void callback(char* topic, byte* message, unsigned int length) {
  Serial.print("Message arrived on topic: ");
  Serial.println(topic);
  Serial.print("Message: ");
  String stMessage;

  for (int i = 0; i < length; i++) {
    Serial.print((char)message[i]);
    stMessage += (char)message[i];
  }

  if (stMessage=="nguy hiem")
  {
    digitalWrite(ledPin, HIGH);
    digitalWrite(relay, HIGH);
    delay(1000);
    digitalWrite(ledPin, LOW);
    digitalWrite(relay, LOW);
    delay(1000);
    tone(buzz,262,1000);
    tone(buzz,0,0);
    tone(buzz,262,1000);
    tone(buzz,0,0);
    tone(buzz,262,1000);
    tone(buzz,0,0);
  }
}
//*

void setup() {
  pinMode(ledPin, OUTPUT);
  pinMode(buzz, OUTPUT);
  pinMode(relay, OUTPUT);

  Serial.begin(115200);
  WIFIConnect();
  client.setServer(MQTTServer, Port);
  client.setCallback(callback);
  WIFIConnect();
  dht.setup(15, DHTesp::DHT22); //Set up loai cam bien la DHT22

  // Set up Oled
  if (!oled.begin(SSD1306_SWITCHCAPVCC, 0x3C)) {
    Serial.println(F("failed to start SSD1306 OLED"));
    while (1);
  }
  //Hien thi tren oled
  delay(2000);         // wait two seconds for initializing
  oled.clearDisplay(); // clear display
  oled.setTextSize(1);         // set text size
  oled.setTextColor(WHITE);    // set text color
  oled.setCursor(0, 2);       // set position to display (x,y)
  // oled.println("KNN_Iot_Module_3"); // set text
  oled.display();              // display on OLED
  
}

void loop() {
  // tone(buzz,262,1000);
  // digitalWrite(ledPin, HIGH);
  // digitalWrite(relay, HIGH);
  // delay(1000);
  // tone(buzz,0,0);
  // digitalWrite(ledPin, LOW);
  // digitalWrite(relay, LOW);
  // delay(1000);

  oled.setCursor(0,2);
  TempAndHumidity data = dht.getTempAndHumidity();
  oled.setTextSize(1);         // set text size
  oled.setTextColor(WHITE);
  oled.println("Temp: "+String(data.temperature)+"oC");
  oled.setCursor(0,10);
  oled.println("Hum: "+String(data.humidity)+"%");
  oled.display();

  if (!client.connected()) {
    MQTT_Reconnect();
  }
  client.loop();
  delay(10); // this speeds up the simulation
}
